# gu_toolkit: Efficient Guide, Compendium, and Applications

This notebook is an opinionated, practical guide for everyday toolkit use. It is organized for **fast onboarding** and **copy/paste utility**.

## How to use this notebook
1. Run the setup cell.
2. Jump to the section you need (quickstart, recipes, applications).
3. Copy a pattern and adapt it.

---

## Contents
- Quickstart
- Frequently used APIs (compendium)
- Parameter and numeric-expression patterns
- Introspection and debugging helpers
- Cool applications (interactive demos)


In [ ]:
# Find where the package root is
import conftest

In [ ]:
# Setup: Import toolkit
from gu_toolkit import *

# 1) Quickstart

## Minimal interactive figure

In [ ]:
fig = Figure(x_range=(-8, 8))
display(fig)

with fig:
    set_title("Quickstart: Interactive sine")
    plot(a*sin(b*x), x, id="wave", label="a*sin(bx)")

## Figure with some configuration

In [ ]:
fig = Figure(x_range=(-3*pi, 3* pi))
display(fig)

with fig:
    set_title("Quickstart: Interacting sine waves")
    plot(sin(b[1]*x), x, id="wave1", label="sin(b_1x)", color="blue", opacity=0.075, thickness=5)
    plot(sin(b[2]*x), x, id="wave2", label="sin(b_2x)", color="green", opacity=0.075, thickness=5)
    plot(sin(b[1]*x)*sin(b[2]*x), x, id="product_wave", label="sin(b_1x) sin(b_2x)", color="red", opacity=1)
    parameter(b[1], value=1, min=1, max=20, step=1)
    parameter(b[2], value=2, min=1, max=20, step=1)
   
# NOTE: width/thickness and alpha/opacity are accepted aliases in plot style kwargs.

## Saving Figure code

Make sure to run the code to create the figure. Play around with it and decide on specific values of the parameters. Later `figure.to_code()` can be used to print out the code that will generate the same figure

In [ ]:
print(fig.to_code())
#BUG make fig.code a readonly property.
#BUG make fig.get_code(options) be the call that allows setting options
#BUG when generating code, call display(fig) instead of just fig. If using context code, use set_title inside the context. Also, display the figure immediately after defintion i.e. display(fig) follows fig=

## Plotting a purely numeric function or a python callable

The callable-first API supports plain Python functions as long as they can evaluate over NumPy arrays.


In [ ]:
import numpy as np

x = SymbolFamily("x")
fig_callable = Figure(x_range=(-6, 6), y_range=(-1.5, 1.5), sampling_points=500)
display(fig_callable)

with fig_callable:
    set_title("Numeric callable: damped oscillation")

    def damped_wave(x):
        return np.exp(-0.15 * x**2) * np.cos(3 * x)

    plot(damped_wave, x, id="damped", color="teal", label="exp(-0.15x^2) cos(3x)")
# BUG
# ---------------------------------------------------------------------------
# KeyError                                  Traceback (most recent call last)
# Cell In[11], line 13
#      10 def damped_wave(x):
#      11     return np.exp(-0.15 * x**2) * np.cos(3 * x)
# ---> 13 plot(damped_wave, x, id="damped", color="teal", label="exp(-0.15x^2) cos(3x)")

# File ~\Documents\notebooks\gu_toolkit\Figure.py:1785, in plot(func, var, parameters, id, label, visible, x_domain, sampling_points, color, thickness, dash, opacity, line, trace, view, vars)
#    1783     fig = Figure()
#    1784     display(fig)
# -> 1785 return fig.plot(
#    1786     func,
#    1787     var,
#    1788     parameters=parameters,
#    1789     id=id,
#    1790     label=label,
#    1791     visible=visible,
#    1792     x_domain=x_domain,
#    1793     sampling_points=sampling_points,
#    1794     color=color,
#    1795     thickness=thickness,
#    1796     dash=dash,
#    1797     line=line,
#    1798     opacity=opacity,
#    1799     trace=trace,
#    1800     view=view,
#    1801     vars=vars,
#    1802 )

# File ~\Documents\notebooks\gu_toolkit\Figure.py:1084, in Figure.plot(self, func, var, parameters, id, label, visible, x_domain, sampling_points, color, thickness, dash, line, opacity, trace, view, vars)
#    1082 if normalized_numeric_fn is not None:
#    1083     plot.set_numeric_function(normalized_var, normalized_numeric_fn, parameters=parameters)
# -> 1084     plot.render()
#    1086 return plot

# File ~\Documents\notebooks\gu_toolkit\figure_plot.py:779, in Plot.render(self, view_id)
#     777 # 3. Compute
#     778 x_values = np.linspace(x_min, x_max, num=int(num))
# --> 779 y_values = np.asarray(self.numeric_expression(x_values))
#     780 self._x_data = x_values.copy()
#     781 self._y_data = y_values.copy()

# File ~\Documents\notebooks\gu_toolkit\numpify.py:318, in NumericFunction.__call__(self, *positional_args, **keyed_args)
#     314     raise ValueError(
#     315         f"Dynamic var {sym!r} ('{var_name}') requires parameter_context at call time"
#     316     )
#     317 if sym not in self._parameter_context:
# --> 318     raise KeyError(
#     319         f"parameter_context is missing symbol {sym!r} ('{var_name}')"
#     320     )
#     321 full_values.append(self._parameter_context[sym])
#     322 continue

# KeyError: "parameter_context is missing symbol x ('x')"
# QueuedDebouncer callback failed
# Traceback (most recent call last):
#   File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\debouncing.py", line 126, in _on_tick
#     self._callback(*call.args, **call.kwargs)
#     ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
#   File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\Figure.py", line 453, in <lambda>
#     lambda *args: self._run_relayout(view_id=view_id, *args),
#                   ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^
#   File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\Figure.py", line 1386, in _run_relayout
#     self.render(reason="relayout")
#     ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^
#   File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\Figure.py", line 1153, in render
#     plot.render(view_id=self.active_view_id)
#     ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#   File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\figure_plot.py", line 779, in render
#     y_values = np.asarray(self.numeric_expression(x_values))
#                           ~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^
#   File "C:\Users\guraltsev\Documents\notebooks\gu_toolkit\numpify.py", line 318, in __call__
#     raise KeyError(
#         f"parameter_context is missing symbol {sym!r} ('{var_name}')"
#     )
# KeyError: "parameter_context is missing symbol x ('x')"

In [ ]:
fig_callable_params = Figure(x_range=(-8, 8), y_range=(-2.5, 2.5), sampling_points=600)
display(fig_callable_params)

with fig_callable_params:
    set_title("Callable with explicit vars + sliders")
    plot(lambda x, A, k: A * np.sin(k * x), x, vars=(x, A, k), id="param-wave", color="orange")
    parameter(A, value=1.2, min=0.0, max=2.5, step=0.1)
    parameter(k, value=1.0, min=0.5, max=4.0, step=0.1)
# BUG same as above

## Quickstart: adding non-trivial info cards

Info cards can mix static HTML with dynamic callback segments that re-evaluate on render and slider changes.
Use this pattern to show diagnostics, derived metrics, and runtime context next to your plot.


In [ ]:
# BUG markdown should contain a more comprehensive description of info card functionality

In [ ]:
fig_info = Figure(x_range=(-2*sp.pi, 2*sp.pi), y_range=(-3, 3), sampling_points=600)
display(fig_info)

with fig_info:
    set_title("Quickstart: plot + diagnostics info cards")
    plot(a*sin(b*x), x, id="main-wave", label="a*sin(bx)", color="royalblue")
    parameter(a, value=1.0, min=0.2, max=2.0, step=0.1)
    parameter(b, value=1.0, min=0.5, max=4.0, step=0.1)

    info("""
    <b>Tips</b><br>
    • Move <code>a</code> to scale amplitude.<br>
    • Move <code>b</code> to change frequency.<br>
    """, id="usage-tips")

    def diagnostics_card(fig, ctx):
        p = fig.params.snapshot()
        amp = p[a]
        freq = p[b]
        period = (2*np.pi/freq) if freq else float("inf")
        est_energy = 0.5 * amp**2
        return (
            f"<b>Wave diagnostics</b><br>"
            f"reason: <code>{ctx.reason}</code><br>"
            f"a={amp:.2f}, b={freq:.2f}<br>"
            f"period≈{period:.3f}, ½a²≈{est_energy:.3f}"
        )

    info(["<hr>", diagnostics_card], id="wave-diagnostics")
# BUG in params.snapshot() allow access by sympy name of symbol if unambiguous. Raise helpful error about using sympy symbols if it does not work. Do not list strings when getting keys() or using an iterator

## Using multiple tabs


In [ ]:
fig_tabs = Figure(x_range=(-6, 6), y_range=(-3, 3), sampling_points=400)
fig_tabs.add_view("phase", x_range=(-2*sp.pi, 2*sp.pi), y_range=(-3, 3), x_label="phase")
display(fig_tabs)

with fig_tabs:
    parameter(a, value=1.0, min=0.0, max=2.0, step=0.1)
    parameter(b, value=0.5, min=-2.0, max=2.0, step=0.1)

    plot(a*sp.sin(x) + b, x, id="shared-wave", view=("main", "phase"), color="crimson")
    with fig_tabs.view("phase"):
        plot(sp.cos(2*x), x, id="phase-only", color="purple")

    info("<b>Shared card:</b> visible on every view", id="shared-card")
    info("<b>Phase note:</b> scoped to the phase tab", id="phase-card", view="phase")


# 2) Compendium: Frequently used operations

### Figure construction and ranges
- `Figure(...)`
- `get_x_range()`, `get_y_range()`
- `set_title(...)`

### Plot and parameter workflow
- `plot(...)`
- `parameter(symbol, ...)`
- `params[symbol]` / `params.snapshot()`
- `render()`

### Numeric expressions
- `numpify(expr, vars=...)`
- `DYNAMIC_PARAMETER` and `UNFREEZE`

### Symbolic parsing
- `parse_latex(latex_string)`


In [ ]:
#BUG: Add significantly more documentation to these commands here


### B) Symbolic-to-numeric with dynamic parameters

Use this when you want to start from symbolic math and still keep interactivity.


In [ ]:
expr = parse_latex(r"A \sin(k x + \phi)")
expr.free_symbols

In [ ]:
f = numpify(expr, vars=['x', 'A', 'k', 'phi'])
f

## 4) Debugging and introspection helpers

These checks are useful when a notebook cell is not behaving as expected.


In [ ]:
with fig3:
    print("Current params:", params.snapshot())
print("Figure has", len(fig3.plots), "plot(s)")
print("Plot ids:", list(fig3.plots.keys()))

plot_obj = fig3.plots["symbolic-wave"]
print("Plot label:", plot_obj.label)
print("Has cached samples:", plot_obj.cached_samples is not None)

#BUG 
#---------------------------------------------------------------------------
#AttributeError                            Traceback (most recent call last)
#Cell In[21], line 8
#      6 plot_obj = fig3.plots["symbolic-wave"]
#      7 print("Plot label:", plot_obj.label)
#----> 8 print("Has cached samples:", plot_obj.cached_samples is not None)
#
#AttributeError: 'Plot' object has no attribute 'cached_samples'